In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv('/content/NCHS_-_Leading_Causes_of_Death__United_States_20251205.csv')

df.head()

,Year,113 Cause Name,Cause Name,State,Deaths,Age-adjusted Death Rate
0,2012,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,21,2.6
1,2017,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,29,3.3
2,2016,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,30,3.7
3,2013,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,30,3.8
4,2000,"Intentional self-harm (suicide) (*U03,X60-X84,...",Suicide,District of Columbia,23,3.8


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10868 entries, 0 to 10867
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Year                     10868 non-null  int64 
 1   113 Cause Name           10868 non-null  object
 2   Cause Name               10868 non-null  object
 3   State                    10868 non-null  object
 4   Deaths                   10868 non-null  object
 5   Age-adjusted Death Rate  10868 non-null  object
dtypes: int64(1), object(5)
memory usage: 509.6+ KB


In [ ]:
# DATA CLEANING
# Standardizing column names: lowercase and replace spaces with underscores
df.columns = df.columns.str.replace(' ', '_').str.replace('-', '_').str.lower()

# FIXING DATA TYPES: Removing commas and converting to numeric

df['deaths'] = pd.to_numeric(df['deaths'].astype(str).str.replace(',', ''),
                             errors='coerce')
df['age_adjusted_death_rate'] = pd.to_numeric(df['age_adjusted_death_rate']
                                              .astype(str).str.replace(',', ''),
                                              errors='coerce')

# Dropping rows where 'deaths' might still be missing
df = df.dropna(subset=['deaths', 'age_adjusted_death_rate'])

# Removed 'United States' as we only wanted individual states for maps
# Removed 'All causes' as we only wanted specific diseases for analysis

df_clean = df[(df['state'] != 'United States') & (df['cause_name'] != 'All causes')]

print(f"Original Rows: \n{len(df)}")
print(f"\nCleaned Rows (States Only, Specific Causes): \n{len(df_clean)}")

Original Rows: 
10868

Cleaned Rows (States Only, Specific Causes): 
9690


In [ ]:
# DESCRIPTIVE STATISTICS

print(df_clean[['deaths', 'age_adjusted_death_rate']].describe())

#LIST OF DISEASES
print(df_clean['cause_name'].unique())

             deaths  age_adjusted_death_rate
count   9690.000000              9690.000000
mean    3744.126213                60.472085
std     7169.937510                67.316423
min       21.000000                 2.600000
25%      536.000000                18.300000
50%     1413.000000                31.450000
75%     3257.750000                55.000000
max    71930.000000               347.400000
['Kidney disease' 'Suicide' 'Influenza and pneumonia'
 "Alzheimer's disease" 'Diabetes' 'CLRD' 'Unintentional injuries' 'Stroke'
 'Heart disease' 'Cancer']


In [ ]:
# DATA AGGREGATION

# 1. Grouping and Summing
agg_by_cause = df_clean.groupby('cause_name')['deaths'].sum().sort_values(ascending=False).reset_index()

print("AGGREGATED DATA: TOTAL DEATHS BY CAUSE (1999-2017)\n")
print(agg_by_cause)

# 2. Grouping and Averaging
agg_by_state = df_clean.groupby('state')['age_adjusted_death_rate'].mean().sort_values(ascending=False).reset_index()

print("\n\nAGGREGATED DATA: HIGHEST MORTALITY STATES (Top 5)\n")
print(agg_by_state.head())

AGGREGATED DATA: TOTAL DEATHS BY CAUSE (1999-2017)

                cause_name    deaths
0            Heart disease  12222640
1                   Cancer  10843644
2                   Stroke   2726523
3                     CLRD   2594927
4   Unintentional injuries   2347820
5      Alzheimer's disease   1494816
6                 Diabetes   1399943
7  Influenza and pneumonia   1094641
8           Kidney disease    858613
9                  Suicide    697016


AGGREGATED DATA: HIGHEST MORTALITY STATES (Top 5)

           state  age_adjusted_death_rate
0    Mississippi                76.635263
1       Kentucky                73.330526
2       Oklahoma                73.298947
3  West Virginia                73.195789
4        Alabama                72.392632


In [ ]:
import plotly.express as px

#  MAP PREPARATION & AGGREGATION

# Create the State Code map (Required for Plotly)
us_state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL',
    'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN',
    'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


df_clean = df_clean.copy()
df_clean['state_code'] = df_clean['state'].map(us_state_abbrev)

# **2. Map Visualization**

In [ ]:
# CHOROPLETH (Average Heart Disease Rate 1999-2017)

df_heart_all = df_clean[df_clean['cause_name'] == 'Heart disease']

# Calculating the AVERAGE death rate per state over all years
df_heart_agg = df_heart_all.groupby(['state_code', 'state'])['age_adjusted_death_rate'].mean().reset_index()

fig1 = px.choropleth(
    df_heart_agg,
    locations='state_code',
    locationmode="USA-states",
    color='age_adjusted_death_rate',
    scope="usa",
    color_continuous_scale="Reds",
    title="Average Heart Disease Death Rate (1999-2017)",
    hover_name='state',
    labels={'age_adjusted_death_rate': 'Avg Death Rate'})

fig1.show()

In [ ]:
# BUBBLE MAP (Total Cancer Deaths 1999-2017)
df_cancer_all = df_clean[df_clean['cause_name'].str.contains('Cancer|Malignant', case=False)]

# Calculating the TOTAL (Sum) deaths per state over all years
df_cancer_agg = df_cancer_all.groupby(['state_code', 'state'])['deaths'].sum().reset_index()

fig2 = px.scatter_geo(
    df_cancer_agg,
    locations="state_code",
    locationmode="USA-states",
    size="deaths",
    color="deaths",
    hover_name="state",
    scope="usa",
    title="Cumulative Cancer Deaths (1999-2017)",
    size_max=50,
    color_continuous_scale="viridis")

fig2.show()

# **3. Aggregation Visualization**

In [ ]:
# BOX PLOT
# Comparing the distribution of death rates for each cause.

boxplot = px.box(
    df_clean,
    x="cause_name",
    y="age_adjusted_death_rate",
    color="cause_name",
    title="Distribution of Death Rates by Cause (Box Plot)",
    labels={"cause_name": "Cause of Death",
            "age_adjusted_death_rate": "Death Rate (Age-Adjusted)"},height=800)

boxplot.update_layout(xaxis_tickangle=-45)


boxplot.show()

In [ ]:
#SCATTER PLOT
# Comparing the total number of Cancer deaths for each state.

df_cancer_all = df_clean[df_clean['cause_name'].str.contains('Cancer|Malignant', case=False)]

# Group by State and Sum the deaths, combined all 18 years of data into one total number per state
df_cancer_total = df_cancer_all.groupby('state')['deaths'].sum().reset_index()
df_cancer_total = df_cancer_total.sort_values('deaths', ascending=False)


fig_scatter = px.scatter(
    df_cancer_total,
    x='state',
    y='deaths',
    title="Design 2: Total Cancer Deaths by State (Scatter Plot)",
    labels={'deaths': 'Total Deaths', 'state': 'State'},
    color='deaths',           # Color dots by intensity
    size='deaths',            # Make dots bigger for higher deaths (optional but looks great)
    color_continuous_scale='Viridis',
    size_max=30)

fig_scatter.update_layout(xaxis_tickangle=-45,height=600, showlegend=False)

fig_scatter.show()

# **4. Interactive Visualization**

In [ ]:
# Aggregated total deaths by cause and year (across all states)
year_cause_summary = (df_clean.groupby(['year', 'cause_name'])['deaths'].sum().reset_index())

interactive = px.bar(
    year_cause_summary,
    x='cause_name',
    y='deaths',
    color='cause_name',
    animation_frame='year',
    title="Total Deaths by Leading Cause Over Time (1999–2017)",
    labels={'cause_name': 'Cause of Death','deaths': 'Total Deaths','year': 'Year'})

interactive.update_layout(
    xaxis_title="Cause of Death",
    yaxis_title="Total Number of Deaths",
    xaxis=dict(categoryorder='total descending'),
    height=600)

interactive.show()


In [ ]:
# Picked one disease for the map
disease_for_map = "Heart disease"

df_hd = df_clean[df_clean['cause_name'] == disease_for_map]

# Average death rate per state per year
df_hd_year_state = (df_hd.groupby(['year', 'state_code', 'state'])['age_adjusted_death_rate'].mean().reset_index())

fig_map_anim = px.choropleth(
    df_hd_year_state,
    locations='state_code',
    locationmode="USA-states",
    color='age_adjusted_death_rate',
    scope="usa",
    color_continuous_scale="Reds",
    hover_name='state',
    animation_frame='year',
    labels={'age_adjusted_death_rate': 'Death Rate'},
    title=f"{disease_for_map} Death Rate by State (1999–2017, Animated)"
)

fig_map_anim.update_layout(
    height=600,
    margin={"r": 0, "t": 50, "l": 0, "b": 0}
)

fig_map_anim.show()
